In [2]:
using JuMP, CPLEX
   
#Crear modelo m y hacer referencia a solver
m = Model(CPLEX.Optimizer)


########################################################
#Sets

#Set de generadores:Hay dos generadores en este sistema
index_generators = ["Generador1","Generador2"]

#Set de periodos: Hay 10 periodos en este sistema
index_periods = 1:10

########################################################
#Parametros

#Cargar datos desde archivo
file = openxl("Datos_Ejemplo.xlsx")

DEMANDA = readxl(file, "Hoja_1!B2:B11")
#En este ejemplo, solo el Generador2 tiene asociado un perfil de disponibilidad por periodo
DISPONIBILIDAD_GENERADOR2 = readxl(file, "Hoja_2!B2:B11")

#Capacidades
K = Dict([("Generador1",400), ("Generador2",800)])
#Costos marginales
MC = Dict([("Generador1",100), ("Generador2",40)])

#Disponibilidad de cada generador por periodo
W=Dict()
for i in index_generators
    for t in index_periods
        #Si es el Generador2, asociamos los valores que habiamos cargado del archivo
        if i=="Generador2"
            W["Generador2",t]=DISPONIBILIDAD_GENERADOR2[t]
        
        #De lo contrario, para el resto de los generadores la disponibilidad por periodo es 1
        else
            W[i,t]=1
        end
    end
end

#Cada periodo dura 4000 horas
H = 4000

#El costo de falla es 500 $/MWh
VOLL= 500

########################################################
#Variables

@variable(m, q[i in index_generators, t in index_periods] >= 0)
@variable(m, u[i in index_periods] >= 0)

########################################################
#Objective function
@objective(m, Min, sum(H*MC[i] * q[i, t] for i in index_generators, t in index_periods)
    +sum(H*VOLL*u[t] for t in index_periods))

########################################################
#Constraints

#Supply=demand
con_oferta_DEMAND=@constraint(m, [t in index_periods], sum(q[i, t] for i in index_generators)+u[t] == DEMANDA[t])

#Max generation limit
con_max_capacidad=@constraint(m, [i in index_generators, t in index_periods], q[i,t]<=K[i]*W[i,t])


#Hasta aqui llega la formulacion del modelo, lo que sigue hacia abajo son scripts para resolverlo y 
#obtener resultados a partir de la solucion optima
######################################################################################################

#Este comando da la orden de resolver el modelo m que formulamos arriba
JuMP.optimize!(m)

#Extraemos el valor optimo de la funcion objetivo
Total_cost=JuMP.objective_value(m)

#Imprimimos el costo total en millones y redondeamos el resultado, de manera que muestre solo 1 decimal
println("Costo total en millones \$: ", round(Total_cost/1000000, digits=1))
println(" ")
println(" ")

#Imprimimos la suma de la generacion de cada tecnologia
#Ojo que para acceder al valor optimo de una variable se usa la funcion value()
println("Generacion anual en GWh")
for i in index_generators
    println(i, " ", round(sum(H*value(q[i,t]) for t in index_periods)/1000,digits=1))
end

println(" ")
println(" ")

#Imprimimos los precios en $/MWh para cada periodo
println("Precios en \$/MWh")
for t in index_periods
    println(t, " ",JuMP.dual(con_oferta_DEMAND[t])/H)
end









MethodError: MethodError: no method matching Model(::Type{CPLEX.Optimizer})
Closest candidates are:
  Model(::Any, !Matched::Symbol, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Int64, !Matched::Array{String,1}, !Matched::Array{String,1}, !Matched::Array{Float64,1}, !Matched::Array{Float64,1}, !Matched::Array{Symbol,1}, !Matched::Array{T,1} where T, !Matched::Array{Tuple{Symbol,Any},1}, !Matched::Any, !Matched::Any, !Matched::Array{Float64,1}, !Matched::Array{Float64,1}, !Matched::Array{Float64,1}, !Matched::Array{Float64,1}, !Matched::Array{Array{Int64,1},1}, !Matched::Array{Array{Tuple{Int64,Int64},1},1}, !Matched::Any, !Matched::MathProgBase.SolverInterface.AbstractMathProgSolver, !Matched::Bool, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Array{T,1} where T, !Matched::JuMP.IndexedVector{Float64}, !Matched::Any, !Matched::Bool, !Matched::Dict{Symbol,Any}, !Matched::IdDict, !Matched::Int64, !Matched::Int64, !Matched::Dict{Symbol,Any}